In [1]:
import os
#os.system('pip uninstall -y transformers')
#os.system('pip uninstall -y tokenizers')
#os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels transformers')
#os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels tokenizers')

# Library

In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import tensorflow as tf

from transformers import  AutoTokenizer, TFBertModel, DebertaConfig, AutoConfig, TFAutoModel, TFRobertaModel
from transformers import RobertaConfig
from transformers import TFDebertaV2Model

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset


import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.12.1
transformers.__version__: 4.20.1
env: TOKENIZERS_PARALLELISM=true


2022-12-27 03:11:47.408488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 03:11:47.409630: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 03:11:47.410313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 03:11:47.412475: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Utils

# Data Loading

In [3]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [4]:
configuration = {
    #"batch_size": BATCH_SIZE,
    #"buffer_size": BUFFER_SIZE,
    #"auto": AUTO,
    #"num_train": NUM_TRAIN,
    "seq_len": 512,
    #"EPOCHS": EPOCHS,
    "model": 'microsoft/deberta-v3-base',
    "n_fold": 5,
    "target_cols" :['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
    "seed": 42    
}

# CV split

In [5]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=configuration['n_fold'], shuffle=True, random_state=configuration['seed'])
for n, (train_index, val_index) in enumerate(Fold.split(train, train[configuration['target_cols']])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    782
1    783
2    782
3    782
4    782
dtype: int64

In [6]:
train

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,1
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,4
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,3
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,1
...,...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5,3
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0,1
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0,3
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5,3


In [7]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    #if isinstance(value, type(tf.constant(0))):
    value = value.tobytes() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [8]:
def make_example(_ids, _masks, _targets):
    ids_feature = Feature(
        bytes_list = BytesList(value = 
                               [tf.io.serialize_tensor(_ids).numpy()]
                              )
                         )

    mask_feature = Feature(
        bytes_list = BytesList(value = 
                             [tf.io.serialize_tensor(_masks).numpy()]
                              )
                         )

    targets      = Feature(
        bytes_list = BytesList(value = 
                             [tf.io.serialize_tensor(_targets).numpy()]
                              )
                         )
    

    features =Features(feature={
        "ids"     : ids_feature,
        "mask": mask_feature,
        "targets"       : targets
                               }
                      )

    example = Example(features  = features)
    return example.SerializeToString()

In [9]:
!pip install -qU swifter

In [10]:
#configuration["seq_len"]
def get_info(train, ind):
    return train.iloc[ind]['ids'], train.iloc[ind]['mask'], train.iloc[ind][['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].to_numpy().astype(np.float32)
#print(len(train['ids'][0]))
#print(len(train['mask'][0]))
#train.iloc[0][['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].to_numpy().astype(float)

def get_tf_info(train, ind):
    ids, mask, labels = get_info(train, ind)
    ids = tf.constant(ids, dtype=tf.int32)
    mask = tf.constant(mask, dtype=tf.int32)
    lables = tf.constant(labels, dtype = tf.float32)
    return ids, mask, labels

In [11]:
#maximum_int = -1
#for i in range(0, len(train)):
#    ids, mask, targets = get_info(train, i)
#    maximum_int = np.max(np.hstack([maximum_int, ids, mask]))
#print(f'Maximum values among ids and mask is {maximum_int}')

In [12]:
#ids, mask, labels = get_info(train, 0)
#ids = tf.constant(ids, dtype=tf.int32)
#mask = tf.constant(mask, dtype=tf.int32)
#lables = tf.constant(labels, dtype = tf.float32)
#example = make_example(ids, mask, labels)

In [13]:
#get_info(train, 0)

In [14]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "ids": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "mask": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        "targets": tf.io.FixedLenFeature([], tf.string)        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    ids = tf.io.parse_tensor(example['ids'], out_type=tf.int32)
    mask = tf.io.parse_tensor(example['mask'], out_type=tf.int32)
    targets = tf.io.parse_tensor(example['targets'], out_type = tf.float32)
    
    #out = tf.cast(example['output'], tf.
    return tf.reshape(ids, 512), tf.reshape(mask, 512), tf.reshape(targets, 6)

#read_labeled_tfrecord(example)

In [15]:
import swifter 

# Write as TFRecord

In [16]:
from tqdm.notebook import tqdm
def tfrecord_generator(model_path):
    model_name = model_path.split('/')[-1]
    N_SPLIT = 5
    #kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    def tokenize(df):
        tokens = tokenizer(df['full_text'], truncation=True,padding="max_length", 
                           max_length=configuration["seq_len"], return_tensors="np")
        df['ids'] = tokens["input_ids"][0]
        df['mask'] = tokens["attention_mask"][0]

        return df[['ids', 'mask']]


    train[['ids', 'mask']] = train.swifter.apply(tokenize, axis = 1)
    for fold in range(0, N_SPLIT):
        print('Writing TFRecord %i of %i...'%(fold+1,N_SPLIT))  
        tmp_train = train.loc[train['fold']==fold]
        with tf.io.TFRecordWriter('feedback_%s_train%d.tfrec'%(model_name, fold)) as writer:
            for i in tqdm(np.arange(0, len(tmp_train))):
                ids, mask, targets= get_tf_info(tmp_train, i)
                example = make_example(ids, mask, targets)
                writer.write(example)
    



In [17]:
tfrecord_generator('microsoft/deberta-v3-base')
tfrecord_generator('allenai/specter')
tfrecord_generator('bert-base-uncased')
tfrecord_generator('xlm-roberta-base')
tfrecord_generator('distilbert-base-uncased-finetuned-sst-2-english')
tfrecord_generator('roberta-base')
tfrecord_generator('bert-base-cased')
tfrecord_generator('distilbert-base-uncased')
tfrecord_generator('Jean-Baptiste/camembert-ner')
tfrecord_generator('roberta-large')
tfrecord_generator('vblagoje/bert-english-uncased-finetuned-pos')
tfrecord_generator('distilroberta-base')
tfrecord_generator('albert-base-v2')
tfrecord_generator('cardiffnlp/twitter-roberta-base-sentiment')
tfrecord_generator('xlm-roberta-large')
tfrecord_generator('facebook/bart-large-mnli')


Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Pandas Apply:   0%|          | 0/3911 [00:00<?, ?it/s]

...

  0%|          | 0/782 [00:00<?, ?it/s]

Writing TFRecord 4 of 5...


  0%|          | 0/782 [00:00<?, ?it/s]

Writing TFRecord 5 of 5...


  0%|          | 0/782 [00:00<?, ?it/s]

In [18]:
n_fold_samples = [782, 783, 782, 782, 782]

In [19]:
f=open('n_fold_samples.txt', 'w')
f.writelines(['782, 783, 782, 782, 782'])
f.close()